<a href="https://colab.research.google.com/github/zama-sarib/Youtube-Comment-Clustering/blob/main/Featurization_%26_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Stemming and Lemmitization

1. Word2vec + Modelling.
2. TfIdf Word2vec + Modelling
3. BERT + Modelling

* Modelling(Kmeans + DBScan + LDA(topic Modelling)
* Metric (Dunn Index +  Sillhoutte Score)
* Optimization Problem ( max Intra-Cluster and min Inter Cluster Distance)



In [53]:
import pandas as pd 
pd.set_option('display.max_columns',None)
import numpy as np
import seaborn as sns
from string import punctuation
import re
import os
from tqdm import tqdm 
import matplotlib.pyplot as plt
import warnings
import missingno as msno
import spacy
import nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import matplotlib as mpl
from collections import Counter
from nltk.corpus import stopwords
nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline
mpl.rcParams['figure.figsize'] = (14, 6)
mpl.rcParams['axes.grid'] = True
sns.set_style("darkgrid")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
data = pd.read_csv("/content/drive/MyDrive/YouTubeComment/basic_feature_data.csv")
print("Number of rows and column: {}".format(data.shape))
data.head()

Number of rows and column: (496265, 8)


,user_id,comment,likes,word_count,char_count,word_density,punc_count,stopwords_count
0,Pankaj Rana,"If this year Asus didn't bring us new 5,000 -...",52.0,70,327,0.214067,15,21
1,Alex Shimla,Only samsung was one vendor that was not push...,0.0,29,126,0.230157,7,11
2,Shubham Sanket Swain,Want to know about updates. By the way does t...,0.0,16,66,0.242421,2,7
3,Aniket Kalbhor,Amazing piece of Tech,0.0,4,18,0.222210,0,1
4,Twitch- xjuiceboxtwitch,JUST BOUGHT BLUE 1,1.0,4,15,0.266649,0,0


# PreProcessing of Data

In [13]:
def preprocess(x):

    SAFE_DIV = 0.0001 
    STOP_WORDS = stopwords.words('english')

    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    
    return x


In [14]:
data['comment'] = data['comment'].apply(lambda x : preprocess(x))
data.head()

,user_id,comment,likes,word_count,char_count,word_density,punc_count,stopwords_count
0,Pankaj Rana,if this year asus did not bring us new 5k 15...,52.0,70,327,0.214067,15,21
1,Alex Shimla,only samsung was one vendor that was not pushi...,0.0,29,126,0.230157,7,11
2,Shubham Sanket Swain,want to know about updates by the way does th...,0.0,16,66,0.242421,2,7
3,Aniket Kalbhor,amazing piece of tech,0.0,4,18,0.222210,0,1
4,Twitch- xjuiceboxtwitch,just bought blue 1,1.0,4,15,0.266649,0,0


In [15]:
data.isna().sum()

user_id            1135
comment               0
likes                 0
word_count            0
char_count            0
word_density          0
punc_count            0
stopwords_count       0
dtype: int64

* Preprocssing the data such as de-contraction of words like don't to donot etc
* Applying Porter Stemmer to the comments for consistency and remove some repetatition of data

## POS tag Count based features.

Abbreviation 	Meaning
* CC 	coordinating conjunction
* CD 	cardinal digit
* DT 	determiner
* EX 	existential there
* FW 	foreign word
* IN 	preposition/subordinating conjunction
* JJ 	This NLTK POS Tag is an adjective (large)
* JJR 	adjective, comparative (larger)
* JJS 	adjective, superlative (largest)
* LS 	list market
* MD 	modal (could, will)
* NN 	noun, singular (cat, tree)
* NNS 	noun plural (desks)
* NNP 	proper noun, singular (sarah)
* NNPS 	proper noun, plural (indians or americans)
* PDT 	predeterminer (all, both, half)
* POS 	possessive ending (parent\ ‘s)
* PRP 	personal pronoun (hers, herself, him, himself)
* PRP$ 	possessive pronoun (her, his, mine, my, our )
* RB 	adverb (occasionally, swiftly)
* RBR 	adverb, comparative (greater)
* RBS 	adverb, superlative (biggest)
* RP 	particle (about)
* TO 	infinite marker (to)
* UH 	interjection (goodbye)
* VB 	verb (ask)
* VBG 	verb gerund (judging)
* VBD 	verb past tense (pleaded)
* VBN 	verb past participle (reunified)
* VBP 	verb, present tense not 3rd person singular(wrap)
* VBZ 	verb, present tense with 3rd person singular (bases)
* WDT 	wh-determiner (that, what)
* WP 	wh- pronoun (who)
* WRB 	wh- adverb (how)






In [20]:
def posTagging(text):
    tokens = nltk.word_tokenize(text)
    tag = nltk.pos_tag(tokens)
    count_dict = Counter(list(map(lambda x: x[1],tag)))
    return count_dict  


dataframe = []
for index,row in tqdm(data.iterrows()):
    dataframe.append(dict(posTagging(row[1])))

temp = pd.DataFrame(dataframe)
temp.fillna(0,inplace=True)
data = pd.concat([data,temp],axis=1)
print("Data rows and columns: {}".format(data.shape))
data.head()

Data rows and columns: (496265, 46)


,user_id,comment,likes,word_count,char_count,word_density,punc_count,stopwords_count,IN,DT,NN,VBD,RB,VB,PRP,JJ,CD,NNS,VBP,VBN,VBZ,MD,PRP$,TO,VBG,WDT,CC,$,WRB,JJS,RP,RBR,WP,NNP,JJR,PDT,FW,RBS,EX,POS,UH,WP$,'',NNPS,SYM,LS
0,Pankaj Rana,if this year asus did not bring us new 5k 15...,52.0,70,327,0.214067,15,21,9.0,5.0,16.0,1.0,6.0,4.0,6.0,6.0,2.0,4.0,6.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alex Shimla,only samsung was one vendor that was not pushi...,0.0,29,126,0.230157,7,11,1.0,2.0,7.0,2.0,6.0,2.0,1.0,2.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Shubham Sanket Swain,want to know about updates by the way does th...,0.0,16,66,0.242421,2,7,2.0,3.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aniket Kalbhor,amazing piece of tech,0.0,4,18,0.222210,0,1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Twitch- xjuiceboxtwitch,just bought blue 1,1.0,4,15,0.266649,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
if not os.path.isfile("/content/drive/MyDrive/YouTubeComment/pos_data.csv"):
    data.to_csv("/content/drive/MyDrive/YouTubeComment/pos_data.csv",index=False)
else:
    print("File already Exist !!")

File already Exist !!


In [30]:
import pandas as pd 
pd.set_option('display.max_columns',None)
import numpy as np
import seaborn as sns
from string import punctuation
import re
import os
from tqdm import tqdm 
import matplotlib.pyplot as plt
import warnings
import missingno as msno
import spacy
import nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import matplotlib as mpl
from collections import Counter
from nltk.corpus import stopwords
nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline
mpl.rcParams['figure.figsize'] = (14, 6)
mpl.rcParams['axes.grid'] = True
sns.set_style("darkgrid")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
data = pd.read_csv("/content/drive/MyDrive/YouTubeComment/pos_data.csv")
print("-----------------------No of data points----------------------")
print("Number of rows and column: {}".format(data.shape))
print()
data.dropna(subset=['comment'],inplace=True,axis=0)
print("-----------------No of data points after dropping--------------")
print("Number of rows and column: {}".format(data.shape))
data.head()

-----------------------No of data points----------------------
Number of rows and column: (496265, 46)

-----------------No of data points after dropping--------------
Number of rows and column: (496246, 46)


,user_id,comment,likes,word_count,char_count,word_density,punc_count,stopwords_count,IN,DT,NN,VBD,RB,VB,PRP,JJ,CD,NNS,VBP,VBN,VBZ,MD,PRP$,TO,VBG,WDT,CC,$,WRB,JJS,RP,RBR,WP,NNP,JJR,PDT,FW,RBS,EX,POS,UH,WP$,'',NNPS,SYM,LS
0,Pankaj Rana,if this year asus did not bring us new 5k 15...,52.0,70,327,0.214067,15,21,9.0,5.0,16.0,1.0,6.0,4.0,6.0,6.0,2.0,4.0,6.0,1.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Alex Shimla,only samsung was one vendor that was not pushi...,0.0,29,126,0.230157,7,11,1.0,2.0,7.0,2.0,6.0,2.0,1.0,2.0,1.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Shubham Sanket Swain,want to know about updates by the way does th...,0.0,16,66,0.242421,2,7,2.0,3.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aniket Kalbhor,amazing piece of tech,0.0,4,18,0.222210,0,1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Twitch- xjuiceboxtwitch,just bought blue 1,1.0,4,15,0.266649,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## TF-IDF Featurization

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)
tfidf = vectorizer.fit_transform(data['comment'])
word2tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [33]:
print("Stop words they were ignored during the training")
print("Number of stop_words that were ignored",len(vectorizer.stop_words_))
print(list(vectorizer.stop_words_)[0:100])

Stop words they were ignored during the training
Number of stop_words that were ignored 61764
['multivitamins', 'vic', 'deception', 'resists', 'എത', 'accord', 'speedcharger', 'dutch', 'onewill', 'wapa', 'schenirio', 'chitter', 'mukka', 'bnata', 'boses', 'starte', 'أحد', 'jaaalimo', 'balmed', 'katane', 'हरब', 'ophbexn79iy', 'tickles', 'companiea', 'colsed', 'highpoint', 'intall', 'tarikey', 'onnnnee', 'aspired', 'triband', 'hibe', 'indeep', 'verywell', 'nair', 'fodder', 'slot1', 'emotionally', 'ako', 'shoulders', 'assemblers', 'loveeeeee', '12series', 'hamkobhi', 'thankyousir', 'एचड', 'paramedical', 'barabari', 'sandhu', 'dual4canect', '92300', 'wrongdoing', 'litt', 'इनम', 'bbett', 'cor150', 'repel', '10rs', 'afterburner', 'muscat', '10mbs', 'purl', 'departmen', 'marcedes', '8cm', 'awasame', 'olwese', 'enike', 'janmasthmi', 'for___', 'darshitnayak', 'du1034tu', '403', 'exchnage', 'anthaledu', '2hour', 'soooper', 'chutiyappa', 'a23', 'sunu', 'olaying', 'cheyan', 'thnksss', 'eios', 'years

## TF-IDF W2Vec featurization

In [ ]:
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
!python -m spacy download en

- After we find TF-IDF scores, we convert each comment to average of word2vec and weighted average of word2vec vectors.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

### Vanilla Word2Vec

In [ ]:
nlp = spacy.load('en_core_web_lg')
vecs = []
comment = data.iloc[1]['comment']
doc = nlp(comment) 
mean_vec = np.zeros([len(doc), len(doc[0].vector)])
for word in doc:
    # word2vec
    vec = word.vector
    mean_vec += vec
mean_vec = mean_vec.mean(axis=0)
vecs.append(mean_vec)


data['comment_feat_w2v'] = list(vecs)

### Word2vec of 'comment'  vectorization

In [49]:
data_comment_feature_tfidf = pd.DataFrame(data.comment_feat_w2v.values.tolist(),index=data.index)
data_comment_feature_tfidf
data.drop(['comment_feat_w2v'],axis=1) #inplace=True

,comment,comment_feat
0,if this year asus did not bring us new 5k 15...,"[-18.208901436068118, 39.889956161379814, -45...."
1,only samsung was one vendor that was not pushi...,"[0.014572754502296448, 22.296855852007866, -6...."
2,want to know about updates by the way does th...,"[3.6124318465590477, 7.384830266237259, -14.96..."
3,amazing piece of tech,"[-3.3368436694145203, 6.871345221996307, 3.020..."
4,just bought blue 1,"[1.1529577746987343, 2.5468208491802216, -1.73..."


### TFIDF wieghted Word2Vec

In [ ]:
nlp = spacy.load('en_core_web_lg')
vecs = []
for comment in tqdm(list(data['comment'])):
    doc = nlp(comment) 
    mean_vec = np.zeros([len(doc), len(doc[0].vector)])
    for word in doc:
        # word2vec
        vec = word.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
    vecs.append(mean_vec)
data['comment_feat_tfidf_w2v'] = list(vecs)

### TFIDF Word2vec of 'comment'  vectorization

In [50]:
data_comment_feature_tfidf_w2v = pd.DataFrame(data.comment_feat_tfidf_w2v.values.tolist(),index=data.index)
data_comment_feature_tfidf_w2v
data.drop(['comment_feat_tfidf_w2v'],axis=1) #inplace=True

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-18.208901,39.889956,-45.993580,-21.850008,38.811571,-41.385244,6.929136,-31.118382,-1.464287,784.857223,-37.525819,-10.707304,20.445061,6.772073,-11.608385,9.085775,-30.431276,394.252965,-58.003877,9.135329,33.522267,32.661153,23.814154,-1.861128,-10.096185,7.398064,-54.513807,-53.030696,35.578849,5.050520,-34.132320,7.890290,-9.339627,23.492581,28.985061,3.120655,14.225209,13.643364,0.859929,-7.776032,-65.092603,27.392775,21.164946,-38.523452,-3.463824,8.162841,-59.859759,-5.767468,-13.823895,15.980272,-28.815008,-1.140160,5.324715,2.503332,1.402823,0.401408,-42.795010,-4.348725,-2.191915,-11.328980,-37.888353,-43.695114,0.596882,59.236967,54.365322,-39.449894,-4.884019,25.431370,36.730972,29.346512,48.457767,59.226471,67.385423,-30.038198,25.742994,33.558210,65.047743,-39.491890,4.467937,100.893624,-19.168519,12.177079,-58.495018,2.393765,8.622213,-2.829667,43.656277,-27.239533,113.061729,-27.244853,-47.800165,16.397627,-18.064510,5.714585,17.633731,16.607361,-8.600191,-47.909805,-1.195900,33.751009,-19.103517,-23.566941,14.228495,-23.743448,37.194132,-350.456894,40.331271,20.109714,10.032200,21.839271,22.944695,-76.443833,26.957712,-13.385973,20.755647,-3.283597,25.272635,-2.974105,-3.412934,36.639211,46.626935,-33.616881,-8.971518,-0.098022,35.389515,14.642467,24.481430,-47.800503,15.279120,-18.116497,18.693276,24.470058,-16.603938,5.909482,46.468035,-4.191758,12.645381,-6.066520,-9.558559,-13.755743,-559.560613,43.451411,42.066174,-39.593770,21.779243,-42.182782,-16.954963,45.864243,-0.680923,-12.336173,32.769719,5.485640,41.320066,2.048080,-19.448346,-28.977428,-32.246766,7.445996,-7.289136,-7.646143,11.177254,72.657660,-5.315825,-27.625031,5.485176,-33.803815,36.015251,-22.531054,61.512382,-2.355903,-21.672453,17.213044,16.842372,-58.785391,-12.945833,16.754733,-6.943485,28.535989,16.949499,-6.887877,10.935394,3.128055,-49.745804,-27.175642,-14.360757,-8.920732,-40.208423,-26.991907,19.991578,34.451128,60.188918,-4.876864,-41.793581,-3.927966,8.917258,6.840076,-40.585948,-39.696581,20.681053,61.737444,28.028480,-56.341063,5.203315,-20.419176,30.952976,4.405656,7.255665,15.581436,33.752253,15.495728,-13.620222,-21.659687,-11.482079,-65.782095,39.512722,36.445657,5.295055,-12.175155,-93.440358,21.887870,-5.501030,-11.433094,-64.399740,13.965763,1.378123,-35.938015,10.699777,37.133816,7.662639,-10.383721,-30.530321,-18.995363,13.722755,49.419817,-2.232814,-22.281841,8.443760,-30.011139,-25.240218,51.685670,19.003798,2.305102,-33.693608,61.794879,27.851678,-97.561529,21.689995,-13.227858,-60.718362,59.138461,-11.802509,-30.045145,-48.151628,3.925868,9.759227,94.892649,-5.575740,28.714212,-33.229982,0.286067,59.734083,70.452035,24.895785,44.622596,13.377625,-33.178274,-2.814550,12.952001,124.758357,43.151019,28.798885,24.838132,-1.483776,-40.245595,-40.225803,-13.275140,-30.799998,33.107857,23.080812,52.643999,-9.992934,26.435700,29.170103,41.868528,-1